In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [2]:
df = pd.read_csv('News_dataset.csv')
df.sample(10)

,id,title,author,text,label
1370,1370,Hillary Clinton introduces Michelle Obama at c...,Doug P.,Hillary Clinton introduces Michelle Obama at c...,1
7136,7136,Donald Trump Says He’s Willing to Talk Directl...,Alan Rappeport,Donald J. Trump proposed another significant s...,0
11299,11299,Breitbart News Daily: Paris Terror - Breitbart,Breitbart News,"On the Friday edition of Breitbart News Daily,...",0
16608,16608,Candidate Replaced by H-1B: Americans Should H...,John Binder,A congressional candidate who formerly worked ...,0
10414,10414,Break Out of the Illusion,NaN,Break Out of the Illusion \nIn the old Sovie...,1
7196,7196,Trump Raises Concern Over Members Of Urban Com...,NaN,Nation Puts 2016 Election Into Perspective By ...,1
12394,12394,Paul Craig Roberts – The Matrix Of Lies And Wh...,King World News,"45 Views October 28, 2016 GOLD , KWN King Worl...",1
20766,20766,President Trump’s Father’s Day Proclamation: D...,Penny Starr,President Donald Trump officially declared tod...,0
10196,10196,"To Create a Quieter City, They’re Recording th...",Emily S. Rueb,"On Thursday, microphones mounted outside two b...",0
6772,6772,So You Think Your Place Is Small? - The New Yo...,Kim Velsey,The first thing that people want to know about...,0


In [3]:
df= df.dropna()

In [4]:

df.reset_index(inplace=True)

In [5]:

df=df.drop(['id','text','author'],axis=1)


In [6]:
df.sample(10)

,index,title,label
15661,17816,"Elizabeth Warren to President Trump: ’Donald, ...",0
3428,3927,Insulin’s Inventor Sold the Patent for $1. The...,1
11995,13642,Nat Geo’s iconic ‘Afghan Girl’ arrested for fa...,1
12986,14771,Get Ready For Civil Unrest: Survey Finds That ...,1
14981,17049,How to Survive a Nuclear Fallout,1
7952,9070,"Terry Jones of Monty Python Has Dementia, His ...",0
12080,13737,Fukushima – The Untouchable Eco-Apocalypse No ...,1
9216,10502,Liberalism Past: the Secret Murder of Postmode...,1
17760,20221,"The Trans-Pacific Partnership is dead, Schumer...",1
6128,6991,"Go Fuck Yourself, Kurt Eichenwald!",1


In [7]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [8]:
import re
tag_re = re.compile(r'<[^>]+>')

def remove_tags(text):
    return tag_re.sub('',text) 

In [10]:
remove_tags(df['title'][0])

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [16]:
from nltk.corpus import stopwords


In [17]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [18]:
len(stopwords.words('english'))

179

In [19]:
def preprocess_text(sen):

    # Step 1: Convert sentence to lower case:
    sentence = sen.lower()

    # Step 2: Remove Html tags
    sentence = remove_tags(sentence)    

    # Step 3: Remove all puntuations and symbols
    sentence = re.sub('[^a-z ]', ' ', sentence)

    # Step 4: Remove single charactersa after removal of punctuations
    sentence = re.sub(r"\s+[a-z]\s+", ' ', sentence)

    # Step 5: Remove extra space
    sentence = re.sub(r'\s+',' ',sentence)

    # Step 6: Remove stopwords
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    sentence = pattern.sub('', sentence)

    return sentence   

In [20]:
X = []
for i in df['title']:
    X.append(preprocess_text(i))

In [21]:
X[0]

'house dem aide even see comey letter jason chaffetz tweeted '

In [22]:
X[1]

'flynn hillary clinton big woman campus breitbart'